In [1]:
import pickle
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder


In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()



,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data = data.drop(['CustomerId','RowNumber','Surname'],axis=1)

In [4]:
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [5]:
onehot_encode_geography=OneHotEncoder(handle_unknown='ignore') 
geo_encoded=onehot_encode_geography.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encode_geography.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [6]:
data =  pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


### Split the data into features and target

In [7]:
X=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']

### Split the data into train and test sets

In [8]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [9]:
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


### Save the encoders and the scaler


In [10]:
with open('label_encoder_gender.pkl','wb') as file:
  pickle.dump(label_encoder_gender,file)

with open('onehot_encode_geography.pkl','wb') as file:
  pickle.dump(onehot_encode_geography,file)

with open('scaler.pkl','wb') as file:
  pickle.dump(scaler,file)


# ANN Regression 

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

## Build the Model

In [12]:
model = Sequential([
  Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
  Dense(32,activation='relu'),
  Dense(1) #output layer for regression
])

2025-09-03 17:42:27.250006: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-09-03 17:42:27.250068: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-09-03 17:42:27.250079: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-09-03 17:42:27.250425: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-03 17:42:27.250997: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## compile the model

In [13]:
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import datetime

#setup the TensorBoard
log_dir = "regressionlogs/fit/" +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)


In [21]:
# Setup Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

## Train the Model

In [22]:
history = model.fit(
  X_train,y_train,
  validation_data=(X_test,y_test),
  epochs=100,
  callbacks=[early_stopping_callback,tensorboard_callback]

)

Epoch 1/100
250/250 [==============================] - 1s 5ms/step - loss: 76058.1250 - mae: 76058.1250 - val_loss: 74428.5781 - val_mae: 74428.5781
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 75706.7422 - mae: 75706.7422 - val_loss: 73841.1875 - val_mae: 73841.1875
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 75345.7188 - mae: 75345.7188 - val_loss: 73470.5781 - val_mae: 73470.5781
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 75029.0312 - mae: 75029.0312 - val_loss: 73385.3594 - val_mae: 73385.3594
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 74630.3750 - mae: 74630.3750 - val_loss: 72886.6094 - val_mae: 72886.6094
Epoch 6/100
250/250 [==============================] - 1s 5ms/step - loss: 74234.6953 - mae: 74234.6953 - val_loss: 72684.0391 - val_mae: 72684.0391
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 74049.1484 - mae: 74049.1484 - 

In [27]:
%reload_ext tensorboard

%tensorboard --logdir regressionlogs/fit/

Reusing TensorBoard on port 6006 (pid 7923), started 0:00:51 ago. (Use '!kill 7923' to kill it.)

## Evaluate Model on test data

In [28]:
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE : {test_mae}')

63/63 [==============================] - 0s 3ms/step - loss: 50034.7812 - mae: 50034.7812
Test MAE : 50034.78125


## Save the model


In [30]:
model.save('regression_model.h5')

/Users/leenakannan/Documents/Skills/Projects/churn_ann/ann/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
